# Mine 2020-21 Data

In [1]:
import requests
import pprint
import os
import json

In [4]:
# Set up the API call variables
game_data = []
season_type = '02' # 02 is regular season
years = ['2015','2016','2017','2018','2019','2020']      # 2020-2021 season
max_game_IDs = [1230,1230,1272,1272,1272,869]  # max number of games in season (changes depending on season)
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', 
    "Upgrade-Insecure-Requests": "1",
    "DNT": "1",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate"
}

# example url
year = years[0]
print('https://statsapi.web.nhl.com/api/v1/game/'
        + year + season_type +str(762).zfill(4)+'/feed/live')

https://statsapi.web.nhl.com/api/v1/game/2015020762/feed/live


In [6]:
# Loop over the counter and format the API call
for i in range(0, len(years)):
    games = {}
    year = years[i]
    max_game_ID = max_game_IDs[i]
    
    for i in range(1,max_game_ID):
        r = requests.get(url='https://statsapi.web.nhl.com/api/v1/game/'
            + year + season_type +str(i).zfill(4)+'/feed/live')
        d = r.json()
#         print(d['gameData']['teams']['home']['name'] + ' vs ' + d['gameData']['teams']['away']['name'] + ' : ' + str(d['gamePk']))

        game_id = d['gamePk']
        home_team = d['gameData']['teams']['home']['name']
        away_team = d['gameData']['teams']['away']['name']
        game_data = []

        for data in d:
            if 'liveData' not in data:
                continue

            single_game = d[data]['plays']['allPlays']
            for play in single_game:
                game_data.append(play)

        games[game_id] = {}
        games[game_id]['home_team'] = home_team
        games[game_id]['away_team'] = away_team
        games[game_id]['events'] = game_data
    print(year + '-' + str(int(year)+1) + ' Games: ' + str(len(games))) # print status message
        
    # save to json file in data directory
    path = os.getcwd()
    path = os.path.abspath(os.path.join(path, os.pardir, 'data/' + year + '_regSeasonPlays.json'))

    with open(path, 'w') as f:
        json.dump(games, f, indent=2)


2015-2016 Games: 1229
2016-2017 Games: 1229
2017-2018 Games: 1271
2018-2019 Games: 1271
2019-2020 Games: 1271
2020-2021 Games: 868


### Count Stats
Just an example of something you can do with this data.

Keep a dictionary containing the number of goals for each player, as well as the number of assists unique to that player's goals.

In [7]:
player_goals = {}

for game_id, game in games.items():
    for play in game['events']:
        if 'result' in play and play['result']['event'] == 'Goal':
            ## extract goal scorer and assiters ##
            goal_scorer = ''
            goal_assisters = []
            for player in play['players']:
                if player['playerType'] == 'Scorer':
                    goal_scorer = player['player']['fullName']
                elif player['playerType'] == 'Assist':
                    goal_assisters.append(player['player']['fullName'])
    #         print(goal_scorer)
    #         print(goal_assisters)

            ## increment player's number of goals ##
            if goal_scorer in player_goals:
                player_goals[goal_scorer]['goals'] += 1
            else:
                player_goals[goal_scorer] = {}
                player_goals[goal_scorer]['goals'] = 1
                player_goals[goal_scorer]['assisters'] = {}

            ## increment assisters for unique to goal scorer's goals ##
            for assister in goal_assisters:
                if assister in player_goals[goal_scorer]['assisters']:
                    player_goals[goal_scorer]['assisters'][assister] += 1
                else:
                    player_goals[goal_scorer]['assisters'][assister] = 1

        
# pprint.pprint(game_data[0])

In [8]:
# pprint.pprint(player_goals)

### Verify Total Points

In [9]:
player_name = 'Adam Fox'
player_name_goals = player_name_goals = player_goals[player_name]['goals']
player_name_assists = 0

for name, value in player_goals.items():
    if player_name in value['assisters']:
        player_name_assists += value['assisters'][player_name]
    
print('Name: ' + player_name)
print('Total Goals: ' + str(player_name_goals))
print('Total Assists: ' + str(player_name_assists))
print('Total Points: ' + str(player_name_goals + player_name_assists))

Name: Adam Fox
Total Goals: 5
Total Assists: 42
Total Points: 47


### How Many Points Due to Interaction?

Want to know how many of Adam Fox's points are due to Panarin

In [10]:
foxGoals_panarinAssists = player_goals['Adam Fox']['assisters']['Artemi Panarin']
panarinGoals_foxAssists = player_goals['Artemi Panarin']['assisters']['Adam Fox']
foxAssists_panarinAssists = 0
for game_id, game in games.items():
    for play in game['events']:
        if 'result' in play and play['result']['event'] == 'Goal':
            ## extract goal scorer and assiters ##
            goal_scorer = ''
            goal_assisters = set()
            for player in play['players']:
                if player['playerType'] == 'Scorer':
                    goal_scorer = player['player']['fullName']
                elif player['playerType'] == 'Assist':
                    goal_assisters.add(player['player']['fullName'])

                if 'Artemi Panarin' in goal_assisters and 'Adam Fox' in goal_assisters:
                    foxAssists_panarinAssists += 1
    #                 print(goal_scorer)
    #                 print(goal_assisters)

print('Times Panarin assists on Fox goals: ' + str(foxGoals_panarinAssists))
print('Times Fox assists on Panarin goals: ' + str(panarinGoals_foxAssists))
print('Times Fox an Panarin assist on a goal: ' + str(foxAssists_panarinAssists))
print('Total Fox and Panarin Points in common: ' + str(foxGoals_panarinAssists + \
                                                          panarinGoals_foxAssists + \
                                                          foxAssists_panarinAssists))

Times Panarin assists on Fox goals: 2
Times Fox assists on Panarin goals: 7
Times Fox an Panarin assist on a goal: 22
Total Fox and Panarin Points in common: 31
